# Optimization in JuMP

Code: https://github.com/JuliaOpt/JuMP.jl Docs: http://www.juliaopt.org/JuMP.jl

From the documentation:

"JuMP is a domain-specific modeling language for mathematical optimization embedded in Julia. It currently supports a number of open-source and commercial solvers (see below) for a variety of problem classes, including linear programming, mixed-integer programming, second-order conic programming, semidefinite programming, and nonlinear programming."
* Simple syntax
* Fast
* Solver independent language
* Supports many solvers
 - Including: Artelys Knitro, Bonmin, Cbc, Clp, Couenne, CPLEX, ECOS, FICO Xpress, GLPK, Gurobi, Ipopt, MOSEK, NLopt, and SCS.
* Written in Julia

Note: Not all solvers support all sorts of sets and constraints. This means that you should select a solver after your requirements. But also that you can use very efficient solvers for special problems, such as LP or QP. 

### In JuMP, each model is connected with an optimizer

In [46]:
using Pkg; Pkg.activate(".")
using JuMP, Ipopt
model = Model(Ipopt.Optimizer)

  Activating project at `~/repos/juliacourse2022/lecture6_optimization_learning`


A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Ipopt

Variables are defined using `@variable` and connected to a model.
Vectors can be defined using `x[1:n]`, and simple constraints can optionally be added

In [47]:
n = 10
@variable(model, x[1:n] >= 0)

10-element Vector{VariableRef}:
 x[1]
 x[2]
 x[3]
 x[4]
 x[5]
 x[6]
 x[7]
 x[8]
 x[9]
 x[10]

### Adding Constraints
Simple constraints can be added using the built in Julia operators, and optionally be given names using

`@constraint(model, [name,] constraint)`

or 

`@constraint(model, [name,] expression in MOI.SomeSetHere())`

In [48]:
@constraint(model, firstbig, 1 <= x[10] <= 10)

firstbig : x[10] ∈ [1.0, 10.0]

And a set of constraints can be created in a simple way

In [49]:
@constraint(model, first3[k=1:3], x[k] <= 10*k)

3-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}}:
 first3[1] : x[1] ≤ 10.0
 first3[2] : x[2] ≤ 20.0
 first3[3] : x[3] ≤ 30.0

Constraints can then be inspected as expected

In [50]:
firstbig

firstbig : x[10] ∈ [1.0, 10.0]

In [51]:
first3

3-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}}:
 first3[1] : x[1] ≤ 10.0
 first3[2] : x[2] ≤ 20.0
 first3[3] : x[3] ≤ 30.0

And the full model so far can be printed

In [52]:
model

A JuMP Model
Feasibility problem with:
Variables: 10
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 3 constraints
`AffExpr`-in-`MathOptInterface.Interval{Float64}`: 1 constraint
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 10 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Ipopt
Names registered in the model: first3, firstbig, x

Vector constraints can be added with the broadcast syntax `.`

In [53]:
using Random
Random.seed!(1)
m = 3

A = randn(m,n)
b = randn(m)

@constraint(model, A*x .== b);

Expressions can be built up using `@expression`

In [54]:
C = randn(m,n)
d = randn(m)

ineq = @expression(model, C*x - d);

And add constraints for expressions in sets

In [55]:
@constraint(model, ineq in MOI.Nonnegatives(m));

Or we could simply have written

`@constraint(model, C*x .<= d)`

### Objective
Simple cost functions can be defined by

`@objective(model, sense, expression)`

In [56]:
M = randn(n,n)
Q = M*M'
p = randn(n)

@objective(model, Min,  x'*Q*x - p'*x);

### Solving
the solver is called using `optimize!`
and values are accessed using `value`

In [57]:
set_optimizer_attribute(model, "print_level", 0)
optimize!(model)

@show termination_status(model)
sol = value.(x)

termination_status(model) = MathOptInterface.LOCALLY_SOLVED


10-element Vector{Float64}:
 -9.957239723111236e-9
  5.997725363269663e-5
 -9.785485135521076e-9
  0.21334244568329733
  3.479068229116638
  2.4360694155676086
  0.26049938814237145
  0.6268970390673398
  1.8832589147125205
  0.999999990035209

We can also ask the solver for a range of data, such as objective_value, dual variables and so on.

See http://www.juliaopt.org/JuMP.jl for details

In [58]:
@show objective_value(model)

@show A*sol-b         # Should be rougly zero

# We can evaluate expressions
@show value.(ineq);    # Should be nonnegative

objective_value(model) = 90.95132936467664
A * sol - b = [0.0, -4.440892098500626e-16, 2.220446049250313e-16]
value.(ineq) = [-9.904256637049969e-9, -9.921460986106467e-9, 1.4300490423847958]


### General nonlinear programming is also supported by some solvers

These objectives and constraints can be entered as

`@NLobjective(model, Min, my_function(x, y))`

and

`@NLconstraint(model, x[1] * x[2] * x[3] * x[4] >= 25)`